In [ ]:
from importlib import reload
#reload(Utilities)

import sys, os
import re

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns
from packaging import version

import itertools
import natsort

import pyodbc
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
import matplotlib.colors as mcolors
import matplotlib.cm as cm #e.g. for cmap=cm.jet
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
import Plot_Box_sns
import GrubbsTest

In [ ]:
df1 = pd.DataFrame([['a', 1, 10], ['b', 2, 20]],
                   columns=['letter', 'number', 'other_number'])

In [ ]:
df2 = pd.DataFrame([[3, 'c'], [4, 'd']],
                   columns=['number', 'letter'])

In [ ]:
df1

In [ ]:
df2

In [ ]:
pd.concat([df1, df2]).fillna(0)

In [ ]:
df2.info()

In [ ]:
conn_outages = Utilities.get_utldb01p_oracle_connection()
conn_aws = Utilities.get_athena_prod_aws_connection()

In [ ]:
schema_name = 'DOVSADM'
table_name = 'DOVS_OUTAGE_FACT'

outages_info_df = Utilities.get_outages_table_info(conn_outages, schema_name, table_name)
outages_info_df

In [ ]:
# natsort.natsorted(outages_info_df['COLUMN_NAME'].unique().tolist())

In [ ]:
schema_name = 'default'
table_name = 'meter_premise'

mp_info_df = Utilities.get_athena_table_info(conn_aws, schema_name, table_name, run_clean=True)
mp_info_df

In [ ]:
schema_name = 'default'
table_name = 'meter_premise_hist'

mp_hist_info_df = Utilities.get_athena_table_info(conn_aws, schema_name, table_name, run_clean=True)
mp_hist_info_df

In [ ]:
set(mp_info_df['col_name']).difference(set(mp_hist_info_df['col_name']))

In [ ]:
set(mp_hist_info_df['col_name']).difference(set(mp_info_df['col_name']))

In [ ]:
schema_name = 'usage_instantaneous'
table_name = 'inst_msr_consume'

mp_info_df = Utilities.get_aws_table_info(conn_aws, schema_name, table_name, run_clean=True)
mp_info_df

In [ ]:
# fields for LaTeX document
col_names = sorted(mp_info_df['col_name'].unique().tolist())
n_rows = int(np.ceil(len(col_names)/3))
col_1_start = 0
col_2_start = n_rows
if len(col_names)%3==0:
    col_3_start = col_2_start+n_rows
else:
    if (len(col_names)-n_rows)%2==0:
        col_3_start = col_2_start+n_rows-1
    else:
        col_3_start = col_2_start+n_rows
#-----
col_1_entries = col_names[col_1_start:col_2_start]
col_2_entries = col_names[col_2_start:col_3_start]
col_3_entries = col_names[col_3_start:]
while len(col_2_entries)<len(col_1_entries):
    col_2_entries.append('')
while len(col_3_entries)<len(col_1_entries):
    col_3_entries.append('')
assert(len(col_1_entries)==len(col_2_entries)==len(col_3_entries))
#-----
for i_row in range(len(col_1_entries)):
    print(f"  {col_1_entries[i_row]} & {col_2_entries[i_row]} & {col_3_entries[i_row]} \\\\ \hline ")

In [ ]:
schema_name = 'instantaneous'
table_name = 'inst_msr_consume'

mp_info_df = Utilities.get_aws_table_info(conn_aws, schema_name, table_name, run_clean=True)
mp_info_df

In [ ]:
# fields for LaTeX document
col_names = sorted(mp_info_df['col_name'].unique().tolist())
n_rows = int(np.ceil(len(col_names)/3))
col_1_start = 0
col_2_start = n_rows
if len(col_names)%3==0:
    col_3_start = col_2_start+n_rows
else:
    if (len(col_names)-n_rows)%2==0:
        col_3_start = col_2_start+n_rows-1
    else:
        col_3_start = col_2_start+n_rows
#-----
col_1_entries = col_names[col_1_start:col_2_start]
col_2_entries = col_names[col_2_start:col_3_start]
col_3_entries = col_names[col_3_start:]
while len(col_2_entries)<len(col_1_entries):
    col_2_entries.append('')
while len(col_3_entries)<len(col_1_entries):
    col_3_entries.append('')
assert(len(col_1_entries)==len(col_2_entries)==len(col_3_entries))
#-----
for i_row in range(len(col_1_entries)):
    print(f"  {col_1_entries[i_row]} & {col_2_entries[i_row]} & {col_3_entries[i_row]} \\\\ \hline ")

In [ ]:
schema_name = 'usage_nonvee'
table_name = 'reading_ivl_nonvee'

mp_info_df = Utilities.get_aws_table_info(conn_aws, schema_name, table_name, run_clean=True)
mp_info_df

In [ ]:
# fields for LaTeX document
col_names = sorted(mp_info_df['col_name'].unique().tolist())
n_rows = int(np.ceil(len(col_names)/3))
col_1_start = 0
col_2_start = n_rows
if len(col_names)%3==0:
    col_3_start = col_2_start+n_rows
else:
    if (len(col_names)-n_rows)%2==0:
        col_3_start = col_2_start+n_rows-1
    else:
        col_3_start = col_2_start+n_rows
#-----
col_1_entries = col_names[col_1_start:col_2_start]
col_2_entries = col_names[col_2_start:col_3_start]
col_3_entries = col_names[col_3_start:]
while len(col_2_entries)<len(col_1_entries):
    col_2_entries.append('')
while len(col_3_entries)<len(col_1_entries):
    col_3_entries.append('')
assert(len(col_1_entries)==len(col_2_entries)==len(col_3_entries))
#-----
for i_row in range(len(col_1_entries)):
    print(f"  {col_1_entries[i_row]} & {col_2_entries[i_row]} & {col_3_entries[i_row]} \\\\ \hline ")

In [ ]:
schema_name = 'meter_events'
table_name = 'end_device_event'

mp_info_df = Utilities.get_aws_table_info(conn_aws, schema_name, table_name, run_clean=True)
mp_info_df

In [ ]:
# natsort.natsorted(mp_info_df['col_name'].unique().tolist())

# --------------------------------------------------------------------------
# --------------------------------------------------------------------------

In [ ]:
def get_sql_outage(date_0, date_1, mjr_mnr_cause):
    if mjr_mnr_cause is None:
        add_where_statement_1 = ''
        add_where_statement_2 = ''
    else:
        add_where_statement_1 = f"AND  DOV.MJR_CAUSE_CD = '{mjr_mnr_cause[0]}'"
        add_where_statement_2 = f"AND  DOV.MNR_CAUSE_CD = '{mjr_mnr_cause[1]}'"
    
    sql_outage = (
    """
    SELECT   DOV.CI_NB, DOV.CMI_NB, DOV.OUTG_REC_NB, 
             DOV.DT_ON_TS, DOV.DT_OFF_TS, 
             DOV.DT_ON_TS - DOV.STEP_DRTN_NB/(60*24) AS DT_OFF_TS_FULL, DOV.STEP_DRTN_NB,
             EXTRACT(YEAR FROM DOV.DT_OFF_TS)  AS START_YEAR,     
             DOV.MJR_CAUSE_CD, DOV.MNR_CAUSE_CD, DOV4.EQUIP_TYP_NM, DOV4.SHORT_NM, 
             DOV.LOCATION_ID, DOV.GIS_CRCT_NB, DOV.STATE_ABBR_TX, 
             PRIM.PREMISE_NB
    FROM     DOVSADM.DOVS_OUTAGE_FACT DOV
             LEFT OUTER JOIN DOVSADM.DOVS_MASTER_GEO_DIM DOV1 ON DOV.OPERATING_UNIT_ID=DOV1.OPRTG_UNT_ID AND DOV.STATE_ABBR_TX=DOV1.STATE_ID AND DOV.OPCO_NBR=DOV1.OPCO_ID AND DOV.DISTRICT_NB=DOV1.DISTRICT_ID AND DOV.SRVC_CNTR_NB=DOV1.AREA_ID AND DOV.GIS_CRCT_NB=DOV1.GIS_CIRCUIT_ID
             LEFT OUTER JOIN DOVSADM.DOVS_OUTAGE_ATTRIBUTES_DIM DOV2 ON DOV.OUTG_REC_NB=DOV2.OUTG_REC_NB
             LEFT OUTER JOIN DOVSADM.DOVS_EQUIPMENT_TYPES_DIM DOV4 ON DOV.EQUIPMENT_CD=DOV4.EQUIPMENT_CD
             LEFT OUTER JOIN DOVSADM.DOVS_OUTAGE_CAUSE_TYPES_DIM DOV5 ON DOV.MJR_CAUSE_CD=DOV5.MJR_CAUSE_CD AND DOV.MNR_CAUSE_CD=DOV5.MNR_CAUSE_CD
             LEFT OUTER JOIN DOVSADM.DOVS_PREMISE_DIM PRIM ON DOV.OUTG_REC_NB=PRIM.OUTG_REC_NB 
    WHERE    DOV.DT_OFF_TS BETWEEN {} AND '{}' {}{}
    AND  DOV2.INTRPTN_TYP_CD = 'S'
    AND  DOV2.CURR_REC_STAT_CD = 'A' 
    AND  DOV.MJR_CAUSE_CD <> 'NI' 
    AND  DOV.DEVICE_CD <> 85 
    AND  DOV.STATE_ABBR_TX = 'OH'
    """
    ).format(f"'{date_0}'", date_1, add_where_statement_1, add_where_statement_2)

    return sql_outage

In [ ]:
date_0_outg = '2018-01-01'
date_1_outg = '2021-12-31'

sql_outage_all = get_sql_outage(date_0_outg, date_1_outg, None)
sql_outage_dl_ol = get_sql_outage(date_0_outg, date_1_outg, ('DL', 'OL'))

# Try to match past queries (1)

In [ ]:
my_meter_premise_info = pd.read_csv(r'C:\Users\s346557\Documents\my_meter_premise_info.csv')
my_meter_premise_info = Utilities_df.remove_prepend_from_columns_in_df(my_meter_premise_info)
my_meter_premise_info_dict = my_meter_premise_info.squeeze().to_dict()

In [ ]:
sql_1_old = (
"""
SELECT   DOV.CI_NB, DOV.CMI_NB, DOV.OUTG_REC_NB, DOV.OUTAGE_NB, DOV.DT_ON_TS, DOV.DT_OFF_TS, 
         DOV.DT_ON_TS - DOV.STEP_DRTN_NB/(60*24) AS DT_OFF_TS_FULL, DOV.STEP_DRTN_NB,
         EXTRACT(YEAR FROM DOV.DT_OFF_TS) AS START_YEAR, 
         DOV.OPERATING_UNIT_ID, DOV.STATE_ABBR_TX, 
         DOV.MJR_CAUSE_CD, DOV.MNR_CAUSE_CD, 
         DOV.LOCATION_ID, DOV.GIS_CRCT_NB, 
         DOV1.OPRTG_UNT_NM, 
         DOV5.MJR_CAUSE_NM, DOV5.MNR_CAUSE_NM, 
         PRIM.OFF_TM, PRIM.REST_TM, PRIM.PREMISE_NB
FROM     DOVSADM.DOVS_OUTAGE_FACT DOV
         LEFT OUTER JOIN DOVSADM.DOVS_MASTER_GEO_DIM DOV1 ON DOV.OPERATING_UNIT_ID=DOV1.OPRTG_UNT_ID AND DOV.STATE_ABBR_TX=DOV1.STATE_ID AND DOV.OPCO_NBR=DOV1.OPCO_ID AND DOV.DISTRICT_NB=DOV1.DISTRICT_ID AND DOV.SRVC_CNTR_NB=DOV1.AREA_ID AND DOV.GIS_CRCT_NB=DOV1.GIS_CIRCUIT_ID
         LEFT OUTER JOIN DOVSADM.DOVS_OUTAGE_ATTRIBUTES_DIM DOV2 ON DOV.OUTG_REC_NB=DOV2.OUTG_REC_NB
         LEFT OUTER JOIN DOVSADM.DOVS_OUTAGE_CAUSE_TYPES_DIM DOV5 ON DOV.MJR_CAUSE_CD=DOV5.MJR_CAUSE_CD AND DOV.MNR_CAUSE_CD=DOV5.MNR_CAUSE_CD
         LEFT OUTER JOIN DOVSADM.DOVS_PREMISE_DIM PRIM ON DOV.OUTG_REC_NB=PRIM.OUTG_REC_NB 
WHERE    PRIM.PREMISE_NB = '{}'
AND  DOV2.INTRPTN_TYP_CD = 'S'
AND  DOV2.CURR_REC_STAT_CD = 'A' 
AND  DOV.MJR_CAUSE_CD <> 'NI' 
AND  DOV.DEVICE_CD <> 85 
AND  DOV.STATE_ABBR_TX = 'OH'
"""
).format(my_meter_premise_info_dict['prem_nb'])

# print(sql_1_old)

In [ ]:
# ****************************************************************
# SELECT STATEMENT
select_stmnt = SQLSelect(field_descs=['CI_NB', 'CMI_NB', 'OUTG_REC_NB', 'OUTAGE_NB', 'DT_ON_TS', 'DT_OFF_TS', 
                                      dict(field_desc='DOV.DT_ON_TS - DOV.STEP_DRTN_NB/(60*24)', alias='DT_OFF_TS_FULL', table_alias_prefix=None), 
                                      'STEP_DRTN_NB', 
                                      dict(field_desc='EXTRACT(YEAR FROM DOV.DT_OFF_TS)', alias='START_YEAR', table_alias_prefix=None), 
                                      'OPERATING_UNIT_ID', 'STATE_ABBR_TX', 'MJR_CAUSE_CD', 'MNR_CAUSE_CD', 'LOCATION_ID', 'GIS_CRCT_NB'], 
                         global_table_alias_prefix='DOV', idxs=None, run_check=True)

select_stmnt.add_select_elements(field_descs=['OPRTG_UNT_NM'], 
                                 global_table_alias_prefix='DOV1', idxs=None, run_check=True)
select_stmnt.add_select_elements(field_descs=['MJR_CAUSE_NM', 'MNR_CAUSE_NM'], 
                                 global_table_alias_prefix='DOV5', idxs=None, run_check=True)
select_stmnt.add_select_elements(field_descs=['OFF_TM', 'REST_TM', 'PREMISE_NB'],
                                 global_table_alias_prefix='PRIM', idxs=None, run_check=True)
select_stmnt = select_stmnt.get_statement_string(include_alias=True, include_table_alias_prefix=True)
print(select_stmnt)

In [ ]:
# ****************************************************************
# FROM AND JOIN STATEMENTS
sql_from = SQLFrom('DOVSADM', 'DOVS_OUTAGE_FACT', 'DOV')
sql_from = DOVSOutages_SQL.add_std_join_DOVS_OUTAGE_FACT_w_DOVS_MASTER_GEO_DIM(add_to_me=sql_from, alias_DOVS_OUTAGE_FACT='DOV', alias_DOVS_MASTER_GEO_DIM='DOV1')
sql_from = DOVSOutages_SQL.add_std_join_DOVS_OUTAGE_FACT_w_DOVS_OUTAGE_ATTRIBUTES_DIM(add_to_me=sql_from, alias_DOVS_OUTAGE_FACT='DOV', alias_DOVS_OUTAGE_ATTRIBUTES_DIM='DOV2')
sql_from = DOVSOutages_SQL.add_std_join_DOVS_OUTAGE_FACT_w_DOVS_OUTAGE_CAUSE_TYPES_DIM(add_to_me=sql_from, alias_DOVS_OUTAGE_FACT='DOV', alias_DOVS_OUTAGE_CAUSE_TYPES_DIM='DOV5')
sql_from = DOVSOutages_SQL.add_std_join_DOVS_OUTAGE_FACT_w_DOVS_PREMISE_DIM(add_to_me=sql_from, alias_DOVS_OUTAGE_FACT='DOV', alias_DOVS_PREMISE_DIM='PRIM')
print(sql_from.get_statement_string())

In [ ]:
# ****************************************************************
# WHERE STATEMENT
sql_where = SQLWhere([dict(field_desc='PREMISE_NB', table_alias_prefix='PRIM', comparison_operator='=', value='{}', needs_quotes=True), 
                      dict(field_desc='INTRPTN_TYP_CD', table_alias_prefix='DOV2', comparison_operator='=', value='S', needs_quotes=True), 
                      dict(field_desc='CURR_REC_STAT_CD', table_alias_prefix='DOV2', comparison_operator='=', value='A', needs_quotes=True), 
                      dict(field_desc='MJR_CAUSE_CD', table_alias_prefix='DOV', comparison_operator='<>', value='NI', needs_quotes=True), 
                      dict(field_desc='DEVICE_CD', table_alias_prefix='DOV', comparison_operator='<>', value=85, needs_quotes=False), 
                      dict(field_desc='STATE_ABBR_TX', table_alias_prefix='DOV', comparison_operator='=', value='OH', needs_quotes=True)], 
                     idxs=None, run_check=True)
where_stmnt = sql_where.get_statement_string()
print(where_stmnt)

In [ ]:
# ****************************************************************
# GROUP BY STATEMENT
sql_groupby = SQLGroupBy()
groupby_stmnt = sql_groupby.get_statement_string()
print(groupby_stmnt)

In [ ]:
# ****************************************************************
# ORDER BY STATEMENT
orderby_statement_coll = SQLOrderBy()
orderby_stmnt = orderby_statement_coll.get_statement_string()
print(orderby_stmnt)

In [ ]:
# ****************************************************************
# FINALIZE
from_and_join_stmnts = sql_from.get_statement_string()
sql_1_new = SQLQuery.combine_sql_components(select_stmnt, from_and_join_stmnts, where_stmnt, 
                                            groupby_stmnt, orderby_stmnt)
sql_1_new = sql_1_new.format(my_meter_premise_info_dict['prem_nb'])
print(sql_1_new)

In [ ]:
df_1_old = pd.read_sql(sql_1_old, conn_outages)
df_1_new = pd.read_sql(sql_1_new, conn_outages)

In [ ]:
print(f'df_1_old.shape = {df_1_old.shape}')
print(f'df_1_new.shape = {df_1_new.shape}')
print(f'df_1_new==df_1_old?: {df_1_new.equals(df_1_old)}')

In [ ]:
df_1_new = df_1_new.sort_values(by=['OUTG_REC_NB'], ignore_index=True)
df_1_old = df_1_old.sort_values(by=['OUTG_REC_NB'], ignore_index=True)
print(f'df_1_new==df_1_old?: {df_1_new.equals(df_1_old)}')

# Try to match past queries (2)

In [ ]:
sql_2_old = (
"""
SELECT   DOV.MJR_CAUSE_CD, DOV.MNR_CAUSE_CD, SUM(DOV.CI_NB) AS SUM_CI_NB, SUM(DOV.CMI_NB) AS SUM_CMI_NB, DOV5.MJR_CAUSE_NM, DOV5.MNR_CAUSE_NM
        
FROM     DOVSADM.DOVS_OUTAGE_FACT DOV
         LEFT OUTER JOIN DOVSADM.DOVS_MASTER_GEO_DIM DOV1 ON DOV.OPERATING_UNIT_ID=DOV1.OPRTG_UNT_ID AND DOV.STATE_ABBR_TX = DOV1.STATE_ID   AND DOV.OPCO_NBR=DOV1.OPCO_ID AND DOV.DISTRICT_NB=DOV1.DISTRICT_ID AND DOV.SRVC_CNTR_NB=DOV1.AREA_ID AND DOV.GIS_CRCT_NB = DOV1.GIS_CIRCUIT_ID
         LEFT OUTER JOIN DOVSADM.DOVS_OUTAGE_ATTRIBUTES_DIM DOV2 ON DOV.OUTG_REC_NB=DOV2.OUTG_REC_NB 
         LEFT OUTER JOIN DOVSADM.DOVS_OUTAGE_CAUSE_TYPES_DIM DOV5 ON DOV.MJR_CAUSE_CD=DOV5.MJR_CAUSE_CD AND DOV.MNR_CAUSE_CD=DOV5.MNR_CAUSE_CD 
WHERE    DOV2.INTRPTN_TYP_CD = 'S' 
AND      DOV2.CURR_REC_STAT_CD = 'A'
AND      DOV.MJR_CAUSE_CD <> 'NI'
AND      DOV.DEVICE_CD <> 85
AND      DOV.JMED_FL = 'N'

GROUP BY DOV.MJR_CAUSE_CD, DOV.MNR_CAUSE_CD, DOV5.MJR_CAUSE_NM, DOV5.MNR_CAUSE_NM
ORDER BY SUM(DOV.CMI_NB) DESC
"""
)
# print(sql_2_old)

In [ ]:
# ****************************************************************
# SELECT STATEMENT
sql_select = SQLSelect(field_descs=['MJR_CAUSE_CD', 'MNR_CAUSE_CD', 
                                    dict(field_desc='SUM(DOV.CI_NB)', alias='SUM_CI_NB', table_alias_prefix=None), 
                                    dict(field_desc='SUM(DOV.CMI_NB)', alias='SUM_CMI_NB', table_alias_prefix=None)], 
                       global_table_alias_prefix='DOV', idxs=None, run_check=True)

sql_select.add_select_elements(field_descs=['MJR_CAUSE_NM', 'MNR_CAUSE_NM'], 
                               global_table_alias_prefix='DOV5', idxs=None, run_check=True)
select_stmnt = sql_select.get_statement_string(include_alias=True, include_table_alias_prefix=True)
print(select_stmnt)

In [ ]:
# ****************************************************************
# FROM AND JOIN STATEMENTS
sql_from = SQLFrom('DOVSADM', 'DOVS_OUTAGE_FACT', 'DOV')
sql_from = DOVSOutages_SQL.add_std_join_DOVS_OUTAGE_FACT_w_DOVS_MASTER_GEO_DIM(add_to_me=sql_from, alias_DOVS_OUTAGE_FACT='DOV', alias_DOVS_MASTER_GEO_DIM='DOV1')
sql_from = DOVSOutages_SQL.add_std_join_DOVS_OUTAGE_FACT_w_DOVS_OUTAGE_ATTRIBUTES_DIM(add_to_me=sql_from, alias_DOVS_OUTAGE_FACT='DOV', alias_DOVS_OUTAGE_ATTRIBUTES_DIM='DOV2')
sql_from = DOVSOutages_SQL.add_std_join_DOVS_OUTAGE_FACT_w_DOVS_OUTAGE_CAUSE_TYPES_DIM(add_to_me=sql_from, alias_DOVS_OUTAGE_FACT='DOV', alias_DOVS_OUTAGE_CAUSE_TYPES_DIM='DOV5')
print(sql_from.get_statement_string())

In [ ]:
# ****************************************************************
# WHERE STATEMENT
sql_where = SQLWhere([dict(field_desc='INTRPTN_TYP_CD', table_alias_prefix='DOV2', comparison_operator='=', value='S', needs_quotes=True), 
                      dict(field_desc='CURR_REC_STAT_CD', table_alias_prefix='DOV2', comparison_operator='=', value='A', needs_quotes=True), 
                      dict(field_desc='MJR_CAUSE_CD', table_alias_prefix='DOV', comparison_operator='<>', value='NI', needs_quotes=True), 
                      dict(field_desc='DEVICE_CD', table_alias_prefix='DOV', comparison_operator='<>', value=85, needs_quotes=False), 
                      dict(field_desc='JMED_FL', table_alias_prefix='DOV', comparison_operator='=', value='N', needs_quotes=True)], 
                     idxs=None, run_check=True)
where_stmnt = sql_where.get_statement_string()
print(where_stmnt)

In [ ]:
# ****************************************************************
# GROUP BY STATEMENT
sql_groupby = SQLGroupBy(field_descs=['MJR_CAUSE_CD', 'MNR_CAUSE_CD'], 
                         global_table_alias_prefix='DOV', idxs=None, run_check=True)
sql_groupby.add_groupby_statements(field_descs=['MJR_CAUSE_NM', 'MNR_CAUSE_NM'], 
                                              global_table_alias_prefix='DOV5', idxs=None, run_check=True)
groupby_stmnt = sql_groupby.get_statement_string()
print(groupby_stmnt)

In [ ]:
# ****************************************************************
# ORDER BY STATEMENT
sql_orderby = SQLOrderBy(['SUM(DOV.CMI_NB)'], global_table_alias_prefix=None, idxs=None, run_check=True)
orderby_stmnt = sql_orderby.get_statement_string()
print(orderby_stmnt)

In [ ]:
# ****************************************************************
# FINALIZE
sql_query = SQLQuery(sql_select=sql_select, 
                     sql_from=sql_from, 
                     sql_where=sql_where, 
                     sql_groupby=sql_groupby, 
                     sql_orderby=sql_orderby)
sql_2_new = sql_query.get_sql_statement()
#print(sql_2_new)

In [ ]:
df_2_old = pd.read_sql(sql_2_old, conn_outages)
df_2_new = pd.read_sql(sql_2_new, conn_outages)

print(f'df_2_old.shape = {df_2_old.shape}')
print(f'df_2_new.shape = {df_2_new.shape}')
print(f'df_2_new==df_2_old?: {df_2_new.equals(df_2_old)}')

In [ ]:
df_2_new = df_2_new.sort_values(by=['MJR_CAUSE_CD', 'MNR_CAUSE_CD'], ignore_index=True)
df_2_old = df_2_old.sort_values(by=['MJR_CAUSE_CD', 'MNR_CAUSE_CD'], ignore_index=True)
print(f'df_2_new==df_2_old?: {df_2_new.equals(df_2_old)}')

# KEEP SOMEWHERE FOR EXAMPLE

In [ ]:
sql_where_1 = SQLWhere([dict(field_desc='aep_opco', comparison_operator='=', value='oh', needs_quotes=True), 
                      dict(field_desc='aep_usage_dt', comparison_operator='BETWEEN', value=['{}','{}'], needs_quotes=True), 
                      dict(field_desc='aep_derived_uom', comparison_operator='=', value='VOLT', needs_quotes=True), 
                      dict(field_desc='aep_srvc_qlty_idntfr', comparison_operator='=', value='AVG', needs_quotes=True), 
                      dict(field_desc='aep_derived_uom', comparison_operator='=', value='KWH', needs_quotes=True), 
                      dict(field_desc='aep_srvc_qlty_idntfr', comparison_operator='=', value='TOTAL', needs_quotes=True)], 
                     idxs=None, run_check=True)

sql_where_2 = SQLWhere([dict(field_desc='aep_opco', comparison_operator='=', value='oh', needs_quotes=True), 
                      dict(field_desc='aep_usage_dt', comparison_operator='BETWEEN', value=['{}','{}'], needs_quotes=True), 
                      dict(field_desc='aep_derived_uom', comparison_operator='=', value='VOLT', needs_quotes=True), 
                      dict(field_desc='aep_srvc_qlty_idntfr', comparison_operator='=', value='AVG', needs_quotes=True), 
                      dict(field_desc='aep_derived_uom', comparison_operator='=', value='KWH', needs_quotes=True), 
                      dict(field_desc='aep_srvc_qlty_idntfr', comparison_operator='=', value='TOTAL', needs_quotes=True)], 
                     idxs=None, run_check=True)

In [ ]:
sql_where_1.combine_where_elements([2,3], 'AND', close_gaps_in_keys=False)
sql_where_1.combine_where_elements([4,5], 'OR', close_gaps_in_keys=False)
sql_where_1.combine_where_elements([2,4], 'OR', close_gaps_in_keys=True)
print(sql_where_1.get_statement_string())

In [ ]:
sql_where_2.combine_where_elements([2,3], 'AND', close_gaps_in_keys=True)
sql_where_2.combine_where_elements([3,4], 'OR', close_gaps_in_keys=False)
sql_where_2.combine_where_elements([2,3], 'OR', close_gaps_in_keys=True)
print(sql_where_2.get_statement_string())

In [ ]:
sql_where_1.get_statement_string()==sql_where_2.get_statement_string()

# END KEEP SOMEWHERE FOR EXAMPLE

# pySpark

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
from pyspark.context import SparkContext, SparkConf
from pyspark.sql import HiveContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import IntegerType, StringType, DateType, TimestampType
from pyspark.sql.types import StructType, StructField
from pyspark.sql.functions import col, lit
import pyspark.sql.functions as func
import sys
import subprocess
from pyspark.sql.functions import unix_timestamp, date_format
from datetime import datetime, timedelta
import boto3

In [ ]:
from ctypes import *
CDLL(r"C:\Users\s346557\Anaconda3\Lib\site-packages\confluent_kafka.libs\librdkafka-5d2e2910.dll")
from confluent_kafka import Consumer, KafkaException, KafkaError

In [ ]:
spark = SparkSession.builder.appName("Basics").getOrCreate()

In [ ]:
sc = spark.sparkContext
sqlctx = SQLContext(sc)

In [ ]:
# server = 'server'
# db = 'db'
# url = 'jdbc:sqlserver//' + server + ';databaseName=' + db
# table = 'table'
# properties = {'driver' : 'com.microsoft.sqlserver.jdbc.SQLServerDriver'}

# df = sqlctx.read.format('jdbc').options(url=url, dbtable=table, driver='{SQL SERVER}').load()

In [ ]:
# df = spark.read \
#        .format("jdbc") \
#        .option("url", "jdbc:postgresql://localhost:5432/TESTDATABASE") \
#        .option("dbtable", "tablename") \
#        .option("user", "username") \
#        .option("password", "password") \
#        .option("driver", "org.postgresql.Driver") \
#        .option("customSchema", "CUSTOMER_ID DECIMAL(38, 0), CUSTOMER_NAME STRING") \ #If You want to Specify Dataframe column
#        .load()


In [ ]:
# zsql="select * from usage_nonvee.reading_ivl_nonvee_hist where aep_opco='oh' and aep_usage_dt>='2021-01-01' and aep_usage_dt<='2021-01-07' and aep_derived_uom='KWH' and serialnumber = '883024464';"

# mydf=sqlctx.sql(zsql)
